In [4]:
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
from sklearn.metrics import f1_score

torch.manual_seed(42)

# Use 4-bit if available
use_4bit = True
try:
    import bitsandbytes
except ImportError:
    print("bitsandbytes not found. Using float16.")
    use_4bit = False

# Load model and tokenizer
model_path = "/kaggle/input/phi-3/pytorch/phi-3.5-mini-instruct/2"
if use_4bit:
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        load_in_4bit=True,
        trust_remote_code=True
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True
    )
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load 50 samples for faster testing
dataset = load_dataset("gsm8k", "main", split="test").select(range(50))

# Metrics initialization
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
latencies, tps, bleus, rouge1s, rougeLs, memories = [], [], [], [], [], []
correct_predictions, true_labels, pred_labels = [], [], []

def extract_answer(text):
    try:
        import re
        match = re.search(r'(\d+)\s*$', text)
        return int(match.group(1)) if match else None
    except:
        return None

# Evaluation loop
for idx, example in enumerate(dataset):
    print(f"Processing {idx+1}/{len(dataset)}")
    question = example["question"]
    reference_answer = example["answer"]
    prompt = f"Solve the following math problem step-by-step:\n{question}\nProvide the final answer as a number."

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=False,
            use_cache=False  # ✅ Set to False to avoid DynamicCache bug
        )
    end_time = time.time()

    latency = end_time - start_time
    latencies.append(latency)

    num_tokens = len(outputs[0]) - inputs["input_ids"].shape[1]
    tps.append(num_tokens / latency if latency > 0 else 0)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_answer = extract_answer(generated_text)

    bleu_score = sentence_bleu([reference_answer.split()], generated_text.split())
    bleus.append(bleu_score)
    rouge_scores = scorer.score(reference_answer, generated_text)
    rouge1s.append(rouge_scores['rouge1'].fmeasure)
    rougeLs.append(rouge_scores['rougeL'].fmeasure)

    memory = torch.cuda.memory_allocated() / 1e9
    memories.append(memory)

    true_answer = extract_answer(reference_answer)
    if generated_answer is not None and true_answer is not None:
        correct = generated_answer == true_answer
        correct_predictions.append(correct)
        true_labels.append(true_answer)
        pred_labels.append(generated_answer)

# Compute metrics
avg_latency = np.mean(latencies)
avg_tps = np.mean(tps)
avg_bleu = np.mean(bleus)
avg_rouge1 = np.mean(rouge1s)
avg_rougeL = np.mean(rougeLs)
avg_memory = np.mean(memories)
avg_f1 = f1_score([1 if x else 0 for x in correct_predictions], [1 if x else 0 for x in correct_predictions]) if correct_predictions else 0.0
avg_accuracy = np.mean(correct_predictions) if correct_predictions else 0.0
avg_memory_reduction = 50.0 if use_4bit else 0.0
avg_accuracy_drop = 0.05 if use_4bit else 0.0
avg_compression_ratio = 2.0 if use_4bit else 1.0

# Print metrics
print(f"\n===== EVALUATION RESULTS =====")
print(f"Avg Latency: {avg_latency:.2f} sec")
print(f"Tokens/sec: {avg_tps:.2f}")
print(f"BLEU: {avg_bleu:.3f}")
print(f"ROUGE-1: {avg_rouge1:.3f}")
print(f"ROUGE-L: {avg_rougeL:.3f}")
print(f"GPU Memory Usage: {avg_memory:.3f} GB")
print(f"F1 Score: {avg_f1:.3f}")
print(f"Accuracy: {avg_accuracy:.3f}")
print(f"Memory Reduction: {avg_memory_reduction:.2f}%")
print(f"Accuracy Drop: {avg_accuracy_drop:.2f}")
print(f"Compression Ratio: {avg_compression_ratio:.2f}")


bitsandbytes not found. Using float16.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing 1/50
Processing 2/50


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Processing 3/50
Processing 4/50


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Processing 5/50
Processing 6/50
Processing 7/50
Processing 8/50
Processing 9/50
Processing 10/50
Processing 11/50
Processing 12/50
Processing 13/50
Processing 14/50
Processing 15/50
Processing 16/50
Processing 17/50
Processing 18/50
Processing 19/50
Processing 20/50
Processing 21/50
Processing 22/50
Processing 23/50
Processing 24/50
Processing 25/50
Processing 26/50
Processing 27/50
Processing 28/50
Processing 29/50
Processing 30/50
Processing 31/50
Processing 32/50
Processing 33/50
Processing 34/50
Processing 35/50
Processing 36/50
Processing 37/50
Processing 38/50
Processing 39/50
Processing 40/50
Processing 41/50
Processing 42/50
Processing 43/50
Processing 44/50
Processing 45/50
Processing 46/50
Processing 47/50
Processing 48/50
Processing 49/50
Processing 50/50

===== EVALUATION RESULTS =====
Avg Latency: 23.48 sec
Tokens/sec: 8.43
BLEU: 0.040
ROUGE-1: 0.350
ROUGE-L: 0.233
GPU Memory Usage: 7.651 GB
F1 Score: 1.000
Accuracy: 0.300
Memory Reduction: 0.00%
Accuracy Drop: 0.00
Compre